In [ ]:
%pip install PyCryptodome

In [ ]:
import requests
from Crypto.Util.Padding import pad, unpad

In [ ]:
asscii_letters = [chr(i).encode() for i in range(32, 127)]

def seperate2blocks(data, block_size):
    return [data[i:i+block_size] for i in range(0, len(data), block_size)]

def encrypt(plaintext):
    return requests.get(f'https://aes.cryptohack.org/ecb_oracle/encrypt/{plaintext}/').json()['ciphertext']

For solving this challenge, we will use the fact that ECB encrypts each block separately and make sure that we could control the content of the block to reveal the flag.
For example if we'll send the following message (in hex):
$$ P = "a"*30 $$
as each block is 32 hex characters (2 hex = 1 byte) we know that the first block of the encrypted message will be:
$$ C = [\underbrace{E("a"*30 + F_1)}_{\text{first block}}, ...] $$
where $F_1$ is the first character of the flag. We can now brute force the first character of the flag by sending plaintext messages of the form $$"a"*30 + c$$ where $c$ is an ascii character, and comparing the results to the first block of $C$ until a match is found.
When $F_1$ is found, we can repeat the process for the second character of the flag as in that case:
$$ C = E("a"*28 + F_1 + F_2) $$
and we can apply the same method again. We can repeat this process until we find $F_1, F_2, ..., F_{15}$. Now we can look at the second block. If we'll send $"a"*32$ as the plaintext, we'll get:
$$ C = [\underbrace{E("a"*32)}_{\text{first block}}, \underbrace{E(F_1...F_{16})}_{\text{second block}},...] $$
and again as we know $F_1, F_2, ..., F_{15}$ we can brute force $F_{16}$. Sending $"a"*30$ will give us:
$$ C = [\underbrace{E("a"*30+F_1)}_{\text{first block}}, \underbrace{E(F_2...F_{17})}_{\text{second block}},...] $$
and so moving by controlling the plaintext we can move block by block until all the flag bytes are found.

In [ ]:
#this code could take some time - be patient
try:
    flag = b''
    for iblock in range(4):
        for i in range(16, 0, -1):
            if iblock==0 and i==16: continue
            brute_force_blocks = {seperate2blocks(encrypt("00"*i + (flag + c).hex()),32)[iblock] : c for c in asscii_letters}
            x = seperate2blocks(encrypt("00"*i), 32)[iblock]
            flag = flag + brute_force_blocks[x]
            print(flag)
except KeyError: #the code will raise an error when the flag is found
    print("The flag is: " + flag.decode())
